# Kaggle connect module

In [ ]:
from google.colab import userdata
import os
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d xavierre/nike-and-adidas-product-list-dataset

Dataset URL: https://www.kaggle.com/datasets/xavierre/nike-and-adidas-product-list-dataset
License(s): MIT
 70% 9.00M/12.9M [00:00<00:00, 17.4MB/s]
100% 12.9M/12.9M [00:00<00:00, 26.0MB/s]


In [ ]:
! unzip "nike-and-adidas-product-list-dataset.zip"

Archive:  nike-and-adidas-product-list-dataset.zip
  inflating: famous brand dataset.xlsx  


# Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

## Data exploration

In [ ]:
df=pd.read_excel("famous brand dataset.xlsx")

In [ ]:
df.head()

,1,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
0,2,Accessories and Equipment,Hats,12231348,Light Menta/Barely Green,Nike Heritage86,APPAREL,IN_STOCK,False,Nike Sportswear,16.95,16.95
1,3,Accessories and Equipment,Hats,12231348,White/Black,Nike Heritage86,APPAREL,IN_STOCK,False,Nike Sportswear,16.95,16.95
2,4,Accessories and Equipment,Hats,12231348,Black/White,Nike Heritage86,APPAREL,IN_STOCK,False,Nike Sportswear,16.95,16.95
3,5,Accessories and Equipment,Hats,13907994,University Blue/White,Nike Heritage86,APPAREL,IN_STOCK,False,Nike Sportswear,12.95,12.95
4,6,Accessories and Equipment,Hats,13907994,Doll/Black,Nike Heritage86,APPAREL,IN_STOCK,False,Nike Sportswear,12.95,12.95


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231114 entries, 0 to 231113
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   1              231114 non-null  int64 
 1   CATEGORY       230709 non-null  object
 2   SUBCATEGORY    230709 non-null  object
 3   PRODUCT_ID     230709 non-null  object
 4   COLOR          230709 non-null  object
 5   PRODUCT_NAME   230712 non-null  object
 6   PRODUCT_TYPE   230709 non-null  object
 7   LABEL          230709 non-null  object
 8   IS_BESTSELLER  229471 non-null  object
 9   BRAND          230927 non-null  object
 10  PRICE_CURRENT  230693 non-null  object
 11  PRICE_RETAIL   230709 non-null  object
dtypes: int64(1), object(11)
memory usage: 21.2+ MB


# Data cleaning

In [ ]:
df.duplicated().count()

231114

There is no duplicates in df

In [ ]:
df = df.drop(columns=[1])
# droping first column

In [ ]:
df.isnull().sum()

,0
CATEGORY,405
SUBCATEGORY,405
PRODUCT_ID,405
COLOR,405
PRODUCT_NAME,402
PRODUCT_TYPE,405
LABEL,405
IS_BESTSELLER,1643
BRAND,187
PRICE_CURRENT,421


We have many columns with the same amount of missing values let`s check this

In [ ]:
df[df['CATEGORY'].isnull()]

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
229530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
229533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
229538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
229541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
229544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
231085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
231088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
231091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN
231094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adidas,NaN,NaN


As we can see all those rows are empty and irrelevant for our analysis

In [ ]:
df.dropna(subset=['CATEGORY'], inplace=True)

In [ ]:
df.isnull().sum()

,0
CATEGORY,0
SUBCATEGORY,0
PRODUCT_ID,0
COLOR,0
PRODUCT_NAME,0
PRODUCT_TYPE,0
LABEL,0
IS_BESTSELLER,1238
BRAND,187
PRICE_CURRENT,16


Checking price columns, maybe we can drop one

In [ ]:
count = df[df['PRICE_CURRENT'] == df['PRICE_RETAIL']].shape[0]
print(f"Number of times price columns are equal: {count}")

Number of times price columns are equal: 180461


Before next steps I want to remove raws with the links

In [ ]:
indices_to_drop = []

for col in df.columns:
    df_links = df[df[col].str.contains('https', na=False)]
    indices_to_drop.extend(df_links.index)

df.drop(indices_to_drop, inplace=True)

Checking non numeric values of PRICE_CURRENT column

In [ ]:
non_numeric_values = df[~df['PRICE_CURRENT'].apply(lambda x: isinstance(x, (int, float)))]['PRICE_CURRENT'].unique()
print(non_numeric_values)

['16.95' '12.95' '17.95' '14.95' '34.95' '29.95' '18.47' '6.95' 'GBP'
 '24.95' '22.95' '54.95' '19.95' '102.95' '38.47' '31.47' '64.95' '32.95'
 '9.95' '44.95' '37.95' '30.47' '35.97' '27.95' '124.95' '15.97' '89.95'
 '72.95' '68.95' '49.95' '104.95' '119.95' '30.95' '69.97' '37.47' '31.95'
 '47.97' '63.97' '38.97' '34.47' '45.47' '51.97' '62.97' '74.95' '84.95'
 '59.95' '34.97' '69.95' '62.95' '41.95' '51.95' '63.95' '26.95' '55.97'
 '43.97' '39.97' '59.47' '19.97' '48.97' '58.95' '99.95' '25.95' '33.97'
 '57.95' '41.97' '76.47' '94.95' '10.47' '24.47' '39.95' '52.47' '134.95'
 '79.97' '32.47' '53.97' '51.47' '44.97' '27.97' '41.47' '50.97' '38.95'
 '47.95' '50.95' '23.97' '29.97' '23.47' '26.47' '58.47' '25.97' '21.95'
 '31.97' '22.47' '23.95' '26.97' '24.97' '42.47' '49.47' '40.47' '42.95'
 '20.95' '28.47' '54.97' '25.47' '28.95' '14.47' '20.97' '45.95' '67.95'
 '33.95' '40.95' '86.95' '79.95' '40.97' '87.95' '46.97' '60.97' '16.47'
 '54.47' '129.95' '73.47' '137.95' '32.97' '90.97'

Removing $ sign and GBP in column

In [ ]:
df['PRICE_CURRENT'] = df['PRICE_CURRENT'].str.replace(r'\s*\$\s*', '', regex=True)
df['PRICE_CURRENT'] = df['PRICE_CURRENT'].replace('GBP', np.nan)
df['PRICE_CURRENT'] = pd.to_numeric(df['PRICE_CURRENT'])

Checking non numeric values of PRICE_RETAIL column

In [ ]:
non_numeric_values = df[~df['PRICE_RETAIL'].apply(lambda x: isinstance(x, (int, float)))]['PRICE_RETAIL'].unique()
print(non_numeric_values)

['16.95' '12.95' '17.95' '14.95' '34.95' '29.95' '22.95' '6.95' '27.95'
 '24.95' '54.95' '19.95' '102.95' '44.95' '64.95' '32.95' '9.95' '37.95'
 '124.95' '89.95' '72.95' '68.95' '49.95' '26.47' '104.95' '119.95'
 '30.95' '99.95' '31.95' '59.95' '79.95' '74.95' '84.95' '69.95' '62.95'
 '41.95' '51.95' '63.95' '26.95' '58.95' '25.95' '57.95' '94.95' '39.95'
 '47.95' '134.95' '71.95' '38.95' '50.95' '21.95' '23.95' '28.95' '42.95'
 '67.95' '20.95' '45.95' '33.95' '40.95' '86.95' '87.95' '129.95' '137.95'
 '30.47' '11.95' '160.95' '18.95' '114.95' '219.95' '264.95' '49.00'
 '13.95' '429.95' '299.95' '15.95' '43.95' '154.95' '10.95' '195.95'
 '36.95' '125.95' '7.95' '95.95' '8.95' '35.95' '164.95' '549.95' '449.95'
 '139.95' '109.95' '169.95' '152.95' '479.95' '719.95' '209.95' '404.95'
 '234.95' '199.95' '144.95' '189.95' '167.95' '314.95' '159.95' '829.95'
 '179.95' '289.95' '112.95' '269.95' '359.95' '214.95' '224.95' '249.95'
 '259.95' '434.95' '247.95' '629.95' '180.00' '244.95' '309.

In [ ]:
df['PRICE_RETAIL'] = pd.to_numeric(df['PRICE_RETAIL'])

After converting price columns to a numeric type I filled empty raws PRICE_CURRENT with values from PRICE_RETAIL

In [ ]:
df['PRICE_CURRENT']=np.where(df['PRICE_CURRENT'].isnull(),df['PRICE_RETAIL'],df['PRICE_CURRENT'])

In [ ]:
df.isnull().sum()

,0
CATEGORY,0
SUBCATEGORY,0
PRODUCT_ID,0
COLOR,0
PRODUCT_NAME,0
PRODUCT_TYPE,0
LABEL,0
IS_BESTSELLER,844
BRAND,187
PRICE_CURRENT,0


Checking IS_BESTSELLER column

In [ ]:
df['IS_BESTSELLER'].value_counts()

,count
IS_BESTSELLER,
False,224382
True,4969
IN_STOCK,118
BEST_SELLER,2


In [ ]:
df[df['IS_BESTSELLER']=='IN_STOCK']

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
120,Accessories and Equipment,Bags and Backpacks,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Particle Grey/Particle Grey/White,19.95,19.95
121,Accessories and Equipment,Bags and Backpacks,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Black/Black/White,19.95,19.95
292,Accessories and Equipment,All Accessories and Equipment,13992764,False,Paris Saint-Germain,Bodysuit and Booties Set,0-6M,IN_STOCK,White,26.47,26.47
293,Accessories and Equipment,All Accessories and Equipment,13992764,False,Paris Saint-Germain,Bodysuit and Booties Set,6-12M,IN_STOCK,White,26.47,26.47
714,Accessories and Equipment,All Accessories and Equipment,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Particle Grey/Particle Grey/White,19.95,19.95
...,...,...,...,...,...,...,...,...,...,...,...
113073,Accessories and Equipment,Bags and Backpacks,14039345,False,Nike Utility Power,51L),ONE SIZE,IN_STOCK,Black/Black/Total Orange,54.95,54.95
113103,Accessories and Equipment,All Accessories and Equipment,14038288,False,Nike Brasilia 9.5,24L),ONE SIZE,IN_STOCK,Iron Grey/Black/White,34.95,34.95
113104,Accessories and Equipment,All Accessories and Equipment,14038288,False,Nike Brasilia 9.5,24L),ONE SIZE,IN_STOCK,Black/Black/White,34.95,34.95
113105,Accessories and Equipment,All Accessories and Equipment,14039345,False,Nike Utility Power,51L),ONE SIZE,IN_STOCK,Black/Black/Total Orange,54.95,54.95


Let`s take out this raws for now

In [ ]:
df_messed_up_columns = df[df['IS_BESTSELLER'] == 'IN_STOCK']
df = df[df['IS_BESTSELLER'] != 'IN_STOCK']

In [ ]:
df[df['IS_BESTSELLER']=='BEST_SELLER']

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
18,Accessories and Equipment,All Accessories and Equipment,13920852,True,Paris Saint-Germain JDI,11L),ONE SIZE,BEST_SELLER,Midnight Navy/White/Midnight Navy,27.95,27.95
119,Accessories and Equipment,Bags and Backpacks,13920852,True,Paris Saint-Germain JDI,11L),ONE SIZE,BEST_SELLER,Midnight Navy/White/Midnight Navy,27.95,27.95


We don`t know to which brand those raws belong so we will drop them


In [ ]:
df = df[df['IS_BESTSELLER'] != 'BEST_SELLER']

In [ ]:
df[df['IS_BESTSELLER'].isna()]

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
229471,Clothing,Clothing,FJ5089,Black,Beach Shorts,Clothing,InStock,NaN,adidas,40.0,40.0
229472,Shoes,Shoes,BC0770,Grey,Five Ten Kestrel Lace Mountain Bike Shoes,Shoes,InStock,NaN,adidas,150.0,150.0
229473,Clothing,Clothing,GC7946,White,Mexico Away Jersey,Clothing,InStock,NaN,adidas,70.0,70.0
229474,Shoes,Shoes,FV4744,Black,Five Ten Hiangle Pro Competition Climbing Shoes,Shoes,InStock,NaN,adidas,160.0,160.0
229475,Clothing,Clothing,GM0239,Blue,Mesh Broken-Stripe Polo Shirt,Clothing,InStock,NaN,adidas,65.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...
231108,Shoes,Shoes,FX2858,White,Supernova+ Shoes,Shoes,InStock,NaN,adidas,120.0,72.0
231110,Shoes,Shoes,H00667,White,Choigo Shoes,Shoes,InStock,NaN,adidas,100.0,70.0
231111,Shoes,Shoes,GZ7705,Black,Daily 3.0 Shoes,Shoes,InStock,NaN,adidas,50.0,35.0
231112,Shoes,Shoes,GZ7706,Pink,Daily 3.0 Shoes,Shoes,InStock,NaN,adidas,50.0,40.0


Let`s fill empty raws with "False" value

In [ ]:
df['IS_BESTSELLER'].fillna(False, inplace=True)

<ipython-input-28-17af70e9c241>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IS_BESTSELLER'].fillna(False, inplace=True)


Dealing with messed up values

In [ ]:
df_messed_up_columns.head()

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
120,Accessories and Equipment,Bags and Backpacks,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Particle Grey/Particle Grey/White,19.95,19.95
121,Accessories and Equipment,Bags and Backpacks,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Black/Black/White,19.95,19.95
292,Accessories and Equipment,All Accessories and Equipment,13992764,False,Paris Saint-Germain,Bodysuit and Booties Set,0-6M,IN_STOCK,White,26.47,26.47
293,Accessories and Equipment,All Accessories and Equipment,13992764,False,Paris Saint-Germain,Bodysuit and Booties Set,6-12M,IN_STOCK,White,26.47,26.47
714,Accessories and Equipment,All Accessories and Equipment,12321510,False,Nike Heritage,1L),ONE SIZE,IN_STOCK,Particle Grey/Particle Grey/White,19.95,19.95


In [ ]:
df_messed_up_columns[['BRAND', 'COLOR']] = df_messed_up_columns[['COLOR', 'BRAND']].values
df_messed_up_columns[['LABEL', 'IS_BESTSELLER']] = df_messed_up_columns[['IS_BESTSELLER', 'LABEL']].values
df_messed_up_columns[['BRAND', 'IS_BESTSELLER']] = df_messed_up_columns[['IS_BESTSELLER', 'BRAND']].values
df_messed_up_columns['BRAND']=np.NAN
# We are setting BRAND column to NAN for future exploration

In [ ]:
df_links.head()

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
229479,f_auto,f_auto,f_auto,q_auto/c1703f01d98847d19348ac4500f9d8e7_9366/T...,"""https://assets.adidas.com/images/w_600",f_auto,f_auto,NaN,adidas,f_auto,q_auto/304055d5fec84445b597ac4500f9c26b_9366/T...
229509,f_auto,f_auto,f_auto,q_auto/49ba5db7ae684824aa87ad210179062d_9366/F...,"""https://assets.adidas.com/images/w_600",f_auto,f_auto,NaN,adidas,f_auto,q_auto/73f363813c78454884a1ad210178bf8d_9366/F...
229511,f_auto,f_auto,f_auto,q_auto/c239721d77d143d58b6cad1f01783970_9366/M...,"""https://assets.adidas.com/images/w_600",f_auto,f_auto,NaN,adidas,f_auto,q_auto/7fa0f04111704bbf9899ad1f01781fae_9366/M...
229521,f_auto,f_auto,f_auto,q_auto/1d22a926eb3e4963a583ad1d011612da_9366/F...,"""https://assets.adidas.com/images/w_600",f_auto,f_auto,NaN,adidas,f_auto,q_auto/2b1bf02513c94d218497ad1d0115fe29_9366/F...
229545,f_auto,f_auto,f_auto,q_auto/e90a20a1c2454d74bc19ad030163757e_9366/B...,"""https://assets.adidas.com/images/w_600",f_auto,f_auto,NaN,adidas,f_auto,q_auto/ad53d57dbfba48fd84d9ad030163666b_9366/B...


In [ ]:
df_links['COLOR'].unique()

array(['q_auto/c1703f01d98847d19348ac4500f9d8e7_9366/Tiro_21_Windbreaker_Black_GP4975_41_detail.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/49ba5db7ae684824aa87ad210179062d_9366/FutureNatural_Shoes_Blue_GX5152_03_standard.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/c239721d77d143d58b6cad1f01783970_9366/Marvel_Superhero_Racer_TR_2.0_Shoes_Black_H04458_03_standard.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/1d22a926eb3e4963a583ad1d011612da_9366/Fluidflash_Shoes_White_GY5019_03_standard.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/e90a20a1c2454d74bc19ad030163757e_9366/Brand_Love_Repeat_Logo_High-Waist_Woven_Pants_Black_GS1355_25_model.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/653591fe417e4b65b261ad1f010b4bc1_9366/Brand_Love_Repeat_Logo_High-Waist_Woven_Pants_Purple_H10244_25_model.jpg~https://assets.adidas.com/images/w_600',
       'q_auto/24c1135cd7b74392bed7ad0b01050298_9366/EQ21_Run_Shoes_Grey_H68075_03_st

Joining fixed columns to df

In [ ]:
df = pd.concat([df, df_messed_up_columns], ignore_index=True)

# Let`s fix Brand Column

In [ ]:
df.isnull().sum()

,0
CATEGORY,0
SUBCATEGORY,0
PRODUCT_ID,0
COLOR,0
PRODUCT_NAME,0
PRODUCT_TYPE,0
LABEL,0
IS_BESTSELLER,0
BRAND,305
PRICE_CURRENT,0


In [ ]:
df[df['BRAND'].isnull()]

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,BRAND,PRICE_CURRENT,PRICE_RETAIL
6909,Clothing,Sport Clothing,12768505,Black,Cleveland Cavaliers Nike Dri-FIT Logo,APPAREL,IN_STOCK,False,NaN,24.95,24.95
6910,Clothing,Sport Clothing,12768505,Black,Cleveland Cavaliers Nike Dri-FIT Logo,APPAREL,IN_STOCK,False,NaN,24.95,24.95
6911,Clothing,Sport Clothing,12768505,Black,Cleveland Cavaliers Nike Dri-FIT Logo,APPAREL,IN_STOCK,False,NaN,24.95,24.95
6912,Clothing,Sport Clothing,12768505,Black,Cleveland Cavaliers Nike Dri-FIT Logo,APPAREL,IN_STOCK,False,NaN,24.95,24.95
12538,Clothing,Tops and T-Shirts,13429557,Fir,Nike (NFL Green Bay Packers),APPAREL,IN_STOCK,False,NaN,24.95,24.95
...,...,...,...,...,...,...,...,...,...,...,...
230308,Accessories and Equipment,Bags and Backpacks,14039345,Black/Black/Total Orange,Nike Utility Power,51L),IN_STOCK,False,NaN,54.95,54.95
230309,Accessories and Equipment,All Accessories and Equipment,14038288,Iron Grey/Black/White,Nike Brasilia 9.5,24L),IN_STOCK,False,NaN,34.95,34.95
230310,Accessories and Equipment,All Accessories and Equipment,14038288,Black/Black/White,Nike Brasilia 9.5,24L),IN_STOCK,False,NaN,34.95,34.95
230311,Accessories and Equipment,All Accessories and Equipment,14039345,Black/Black/Total Orange,Nike Utility Power,51L),IN_STOCK,False,NaN,54.95,54.95


In [ ]:
df['BRAND'].value_counts()

,count
BRAND,
Nike,111626
Nike Sportswear,72548
Jordan,29395
NIKEiD,7318
NikeLab,4320
Nike Pro,3935
adidas,844
ACG,22


Rename BRAND column and creating new one for better understanding

In [ ]:
df = df.rename(columns={'BRAND': 'SUB_BRAND'})

Let`s try to take out SUB_BRAND values from PRODUCT_NAME

In [ ]:
unique_sub_brands = df['SUB_BRAND'].dropna().unique()
for sub_brand in unique_sub_brands:
    df.loc[df['SUB_BRAND'].isna() & df['PRODUCT_NAME'].str.contains(sub_brand, case=False), 'SUB_BRAND'] = sub_brand

In [ ]:
df['SUB_BRAND'].isna().sum()

4

In [ ]:
df['SUB_BRAND'].value_counts()

,count
SUB_BRAND,
Nike,111898
Nike Sportswear,72552
Jordan,29420
NIKEiD,7318
NikeLab,4320
Nike Pro,3935
adidas,844
ACG,22


In [ ]:
df['SUB_BRAND'].fillna('Adidas', inplace=True)

In [ ]:
df['BRAND'] = np.where(df['SUB_BRAND'].isin(['Nike', 'Jordan', 'NIKEiD', 'NikeLab', 'Nike Sportswear', 'Nike Pro', 'ACG']), 'Nike', 'Adidas')

In [ ]:
df['BRAND'].value_counts()

,count
BRAND,
Nike,229465
Adidas,848


In [ ]:
df.isnull().sum()

,0
CATEGORY,0
SUBCATEGORY,0
PRODUCT_ID,0
COLOR,0
PRODUCT_NAME,0
PRODUCT_TYPE,0
LABEL,0
IS_BESTSELLER,0
SUB_BRAND,0
PRICE_CURRENT,0


Let`s check unique values in columns

In [ ]:
df['CATEGORY'].unique()

array(['Accessories and Equipment', 'Clothing', 'Shoes', 'Accessories'],
      dtype=object)

In [ ]:
df['CATEGORY'] = df['CATEGORY'].replace({'Accessories': 'Accessories and Equipment'})

In [ ]:
df['SUBCATEGORY'].unique()

array(['Hats', 'Bags and Backpacks', 'All Accessories and Equipment',
       'Socks', 'Skirts and Dresses', 'Sports Bras', 'Jackets',
       'Kits and Jerseys', 'Sport Clothing', 'Tracksuits',
       'Trousers and Leggings', 'Hoodies and Sweatshirts', 'Shorts',
       'Tops and T-Shirts', 'All Clothing', 'Running', 'Football',
       'Jordan', 'Lifestyle', 'All Shoes', 'Swimwear', 'Matching Sets',
       'Trousers', 'Leggings', 'Nike By You', 'Training and Gym',
       'Trousers and Tights', 'Skateboarding', 'Basketball', 'Clothing',
       'Shoes', 'Accessories'], dtype=object)

In [ ]:
df['COLOR'].unique()

array(['Light Menta/Barely Green', 'White/Black', 'Black/White', ...,
       'Silver', 'Brown', 'Medium Olive/Black/Particle Grey'],
      dtype=object)

In [ ]:
df['PRODUCT_TYPE'].unique()

array(['APPAREL', 'EQUIPMENT', 'FOOTWEAR', 'ACCESSORIES', 'Clothing',
       'Bags and Backpacks', ' Bodysuit and Booties Set',
       'All Accessories and Equipment', ' Headband and Bib Set'],
      dtype=object)

In [ ]:
allowed_product_types = ['APPAREL', 'EQUIPMENT', 'FOOTWEAR', 'ACCESSORIES', 'Clothing',  'Shoes', 'Accessories',  ' Bodysuit and Booties Set',   ' Headband and Bib Set']

for index, row in df[~df['PRODUCT_TYPE'].isin(allowed_product_types)].iterrows():
  df.loc[index, 'PRODUCT_TYPE'] = row['SUBCATEGORY']

In [ ]:
df['PRODUCT_TYPE'] = df['PRODUCT_TYPE'].replace({'Accessories': 'ACCESSORIES', 'Shoes': 'FOOTWEAR'})

In [ ]:
df['LABEL'].unique()

array(['IN_STOCK', 'BEST_SELLER', 'JUST_IN', 'SOLD_OUT', 'COMING_SOON',
       'nikePlusExclusive', 'CUSTOMIZABLE', 'InStock', 'OutOfStock'],
      dtype=object)

In [ ]:
df['LABEL'] = df['LABEL'].replace({'InStock': 'IN_STOCK', 'OutOfStock': 'SOLD_OUT'})

In [ ]:
df[df['LABEL']=='BEST_SELLER']

,CATEGORY,SUBCATEGORY,PRODUCT_ID,COLOR,PRODUCT_NAME,PRODUCT_TYPE,LABEL,IS_BESTSELLER,SUB_BRAND,PRICE_CURRENT,PRICE_RETAIL,BRAND
7,Accessories and Equipment,Bags and Backpacks,14039281,Black/Black/White,Nike Brasilia,EQUIPMENT,BEST_SELLER,True,Nike,34.95,34.95,Nike
8,Accessories and Equipment,Bags and Backpacks,12632875,Black/Black/White,Nike Brasilia,EQUIPMENT,BEST_SELLER,True,Nike,29.95,29.95,Nike
16,Accessories and Equipment,All Accessories and Equipment,14039281,Black/Black/White,Nike Brasilia,EQUIPMENT,BEST_SELLER,True,Nike,34.95,34.95,Nike
18,Accessories and Equipment,All Accessories and Equipment,12632942,Football Grey/University Blue/Bright Crimson,Nike Classic,EQUIPMENT,BEST_SELLER,True,Nike,24.95,24.95,Nike
19,Accessories and Equipment,All Accessories and Equipment,12632942,Black/Black/White,Nike Classic,EQUIPMENT,BEST_SELLER,True,Nike,24.95,24.95,Nike
...,...,...,...,...,...,...,...,...,...,...,...,...
228009,Shoes,All Shoes,14114130,White/Solar Flare,Nike Air Force 1 '07,FOOTWEAR,BEST_SELLER,True,Nike Sportswear,114.95,114.95,Nike
228010,Shoes,All Shoes,14114130,White/Solar Flare,Nike Air Force 1 '07,FOOTWEAR,BEST_SELLER,True,Nike Sportswear,114.95,114.95,Nike
228012,Shoes,All Shoes,14114130,White/Solar Flare,Nike Air Force 1 '07,FOOTWEAR,BEST_SELLER,True,Nike Sportswear,114.95,114.95,Nike
228014,Shoes,All Shoes,14114130,White/Solar Flare,Nike Air Force 1 '07,FOOTWEAR,BEST_SELLER,True,Nike Sportswear,114.95,114.95,Nike


Let`s check price outliers

In [ ]:
df.describe()

,PRICE_CURRENT,PRICE_RETAIL
count,230313.000000,230313.000000
mean,75.164093,79.666381
std,53.710436,55.223144
min,6.950000,6.950000
25%,37.950000,39.950000
50%,59.950000,64.950000
75%,109.950000,109.950000
max,1337.900000,2229.950000


In [ ]:
numeric_columns = ['PRICE_CURRENT', 'PRICE_RETAIL']

for col in numeric_columns:
  fig=px.box(df,x=col, color='BRAND', facet_col='CATEGORY')
  fig.show()

In [ ]:
df.to_csv('clean_data.csv', index=False)

In [ ]:
nike_df=df[df['BRAND']=='Nike']

In [ ]:
nike_df.to_csv('nike_data.csv', index=False)

More data

In [ ]:
!kaggle datasets download -d jessicarippman/nike-and-adidas-footwear-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicarippman/nike-and-adidas-footwear-dataset
License(s): unknown
  0% 0.00/54.9k [00:00<?, ?B/s]
100% 54.9k/54.9k [00:00<00:00, 47.6MB/s]


In [ ]:
! unzip "nike-and-adidas-footwear-dataset.zip"

Archive:  nike-and-adidas-footwear-dataset.zip
  inflating: nike_adidas.csv         


In [ ]:
extra_df=pd.read_csv("nike_adidas.csv")

In [ ]:
extra_df.head()

,brand,product_id,product_name,department,price_category,listing_price,sale_price,discount,revenue
0,Adidas,G27341,Women's adidas Originals Sleek Shoes,Women,Competitve,75.99,37.99,0.5,1641.17
1,Adidas,CM0081,Women's adidas Swim Puka Slippers,Women,Economy,9.99,5.99,0.4,398.93
2,Adidas,B44832,Women's adidas Sport Inspired Questar Ride Shoes,Women,Competitve,69.99,34.99,0.5,2204.37
3,Adidas,D98205,Women's adidas Originals Taekwondo Shoes,Women,Competitve,79.99,39.99,0.5,5182.70
4,Adidas,B75586,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Women,Economy,47.99,19.20,0.6,1555.20


In [ ]:
extra_df['brand'].value_counts()

,count
brand,
Adidas,2575
Nike,545


# Uploading Sales

In [ ]:
!kaggle datasets download -d sagarmorework/adidas-us-sales

Dataset URL: https://www.kaggle.com/datasets/sagarmorework/adidas-us-sales
License(s): MIT
  0% 0.00/146k [00:00<?, ?B/s]
100% 146k/146k [00:00<00:00, 74.2MB/s]


In [ ]:
! unzip "adidas-us-sales.zip"

Archive:  adidas-us-sales.zip
  inflating: Adidas US Sales.csv     


In [ ]:
adidas_sales_df=pd.read_csv("Adidas US Sales.csv")

In [ ]:
! kaggle datasets download -d ahmadezzat1/nike-sales-nti15

Dataset URL: https://www.kaggle.com/datasets/ahmadezzat1/nike-sales-nti15
License(s): apache-2.0
  0% 0.00/76.3k [00:00<?, ?B/s]
100% 76.3k/76.3k [00:00<00:00, 50.9MB/s]


In [ ]:
! unzip "nike-sales-nti15.zip"

Archive:  nike-sales-nti15.zip
  inflating: Nike Dataset.csv        


In [ ]:
nike_sales_df=pd.read_csv("Nike Dataset.csv")

In [ ]:
adidas_sales_df.head()

,sales_id,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,1,Foot Locker,1185732,2021-11-06,Southeast,South Carolina,Charleston,Men's Street Footwear,50,278,13900,792,Online
1,2,Foot Locker,1185732,2020-03-14,Northeast,New York,New York,Men's Apparel,46,278,12788,435,Outlet
2,3,West Gear,1128299,2021-01-19,West,California,San Francisco,Men's Athletic Footwear,47,278,13066,287,Outlet
3,4,Walmart,1128299,2021-06-17,Southeast,Florida,Orlando,Women's Apparel,103,218,22454,1257,Online
4,5,Sports Direct,1197831,2021-09-14,South,Alabama,Birmingham,Women's Street Footwear,36,278,10008,530,Outlet


In [ ]:
nike_sales_df.head()

,Invoice Date,Product,Region,Retailer,Sales Method,State,Price per Unit,Total Sales,Units Sold
0,01-01-2020,Men's Street Footwear,Northeast,Foot Locker,In-store,New York,50,6000,120
1,02-01-2020,Men's Athletic Footwear,Northeast,Foot Locker,In-store,New York,50,5000,100
2,03-01-2020,Women's Street Footwear,Northeast,Foot Locker,In-store,New York,40,4000,100
3,04-01-2020,Women's Athletic Footwear,Northeast,Foot Locker,In-store,New York,45,3825,85
4,05-01-2020,Men's Apparel,Northeast,Foot Locker,In-store,New York,60,5400,90


Let`s check datsets

In [ ]:
adidas_sales_df.isnull().sum()

,0
sales_id,0
retailer,0
retailer_id,0
invoice_date,0
region,0
state,0
city,0
product,0
price_per_unit,0
units_sold,0


In [ ]:
nike_sales_df.isnull().sum()

,0
Invoice Date,0
Product,0
Region,0
Retailer,0
Sales Method,0
State,0
Price per Unit,0
Total Sales,0
Units Sold,0


In [ ]:
adidas_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9637 entries, 0 to 9636
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sales_id          9637 non-null   int64 
 1   retailer          9637 non-null   object
 2   retailer_id       9637 non-null   int64 
 3   invoice_date      9637 non-null   object
 4   region            9637 non-null   object
 5   state             9637 non-null   object
 6   city              9637 non-null   object
 7   product           9637 non-null   object
 8   price_per_unit    9637 non-null   int64 
 9   units_sold        9637 non-null   int64 
 10  total_sales       9637 non-null   int64 
 11  operating_profit  9637 non-null   int64 
 12  sales_method      9637 non-null   object
dtypes: int64(6), object(7)
memory usage: 978.9+ KB


In [ ]:
nike_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Invoice Date    9360 non-null   object
 1   Product         9360 non-null   object
 2   Region          9360 non-null   object
 3   Retailer        9360 non-null   object
 4   Sales Method    9360 non-null   object
 5   State           9360 non-null   object
 6   Price per Unit  9360 non-null   int64 
 7   Total Sales     9360 non-null   int64 
 8   Units Sold      9360 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 658.2+ KB


Let`s rename columns for better merging

In [ ]:
adidas_sales_df = adidas_sales_df.rename(columns=
 {'retailer': 'Retailer',
  'invoice_date': 'Invoice Date',
  'product': 'Product',
  'price_per_unit': 'Price per Unit',
  'units_sold': 'Units Sold',
  'total_sales': 'Total Sales',
  'state': 'State',
  'region': 'Region',
  'sales_method': 'Sales Method'})

Let`s drop irrelevant columns and add brand column

In [ ]:
adidas_sales_df = adidas_sales_df.drop(columns=['retailer_id', 'sales_id', 'city','operating_profit'])
adidas_sales_df['Brand'] = 'Adidas'
nike_sales_df['Brand'] = 'Nike'

Let`s fix date column

In [ ]:
adidas_sales_df['Invoice Date'].unique()

array(['2021-11-06', '2020-03-14', '2021-01-19', '2021-06-17',
       '2021-09-14', '2021-07-06', '2021-01-12', '2020-01-21',
       '2020-08-17', '2021-07-16', '2021-08-25', '2021-03-02',
       '2021-08-23', '2021-08-27', '2021-08-21', '2021-01-11',
       '2021-11-17', '2021-01-23', '2021-04-03', '2020-03-10',
       '2021-01-18', '2021-04-18', '2021-09-19', '2021-05-12',
       '2021-10-11', '2021-03-16', '2021-10-21', '2021-02-21',
       '2021-03-19', '2021-11-13', '2021-02-11', '2021-01-09',
       '2021-02-09', '2021-06-11', '2021-06-06', '2021-04-10',
       '2021-09-07', '2021-03-09', '2021-09-16', '2021-09-11',
       '2020-05-03', '2020-04-19', '2021-02-15', '2020-04-27',
       '2021-03-13', '2021-03-10', '2021-01-10', '2020-04-28',
       '2021-01-06', '2021-11-11', '2021-02-12', '2021-05-25',
       '2021-09-24', '2020-08-06', '2021-06-07', '2020-08-11',
       '2020-09-11', '2020-11-13', '2020-01-27', '2020-03-04',
       '2020-03-18', '2020-01-25', '2021-04-23', '2021-

In [ ]:
nike_sales_df['Invoice Date'].unique()

array(['01-01-2020', '02-01-2020', '03-01-2020', '04-01-2020',
       '05-01-2020', '06-01-2020', '07-01-2020', '08-01-2020',
       '21-01-2020', '22-01-2020', '23-01-2020', '24-01-2020',
       '25-01-2020', '26-01-2020', '27-01-2020', '28-01-2020',
       '29-01-2020', '30-01-2020', '31-01-2020', '01-02-2020',
       '02-02-2020', '03-02-2020', '04-02-2020', '05-02-2020',
       '06-02-2020', '07-02-2020', '08-02-2020', '09-02-2020',
       '10-02-2020', '03-03-2020', '04-03-2020', '05-03-2020',
       '06-03-2020', '07-03-2020', '08-03-2020', '09-03-2020',
       '10-03-2020', '11-03-2020', '12-03-2020', '13-03-2020',
       '14-03-2020', '15-03-2020', '16-03-2020', '17-03-2020',
       '18-03-2020', '31-03-2020', '17-04-2020', '18-04-2020',
       '19-04-2020', '20-04-2020', '21-04-2020', '22-04-2020',
       '23-04-2020', '24-04-2020', '25-04-2020', '26-04-2020',
       '27-04-2020', '28-04-2020', '29-04-2020', '30-04-2020',
       '01-05-2020', '02-05-2020', '03-05-2020', '04-05

In [ ]:
adidas_sales_df['Invoice Date'] = pd.to_datetime(adidas_sales_df['Invoice Date'], format='%Y-%m-%d')
nike_sales_df['Invoice Date'] = pd.to_datetime(nike_sales_df['Invoice Date'], format='%d-%m-%Y')

In [ ]:
nike_sales_df['Invoice Date'] = pd.to_datetime(nike_sales_df['Invoice Date'], errors='coerce')
adidas_sales_df['Invoice Date'] = pd.to_datetime(adidas_sales_df['Invoice Date'], errors='coerce')

Joining in one df

In [ ]:
sales_df=pd.concat([adidas_sales_df,nike_sales_df],ignore_index=True)

In [ ]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18997 entries, 0 to 18996
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Retailer        18997 non-null  object        
 1   Invoice Date    18997 non-null  datetime64[ns]
 2   Region          18997 non-null  object        
 3   State           18997 non-null  object        
 4   Product         18997 non-null  object        
 5   Price per Unit  18997 non-null  int64         
 6   Units Sold      18997 non-null  int64         
 7   Total Sales     18997 non-null  int64         
 8   Sales Method    18997 non-null  object        
 9   Brand           18997 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 1.4+ MB


In [ ]:
sales_df.to_csv('sales_data.csv', index=False)

In [ ]:
fig = px.box(nike_sales_df, x='Units Sold', y='Product', color='Brand')
fig.show()

In [ ]:
fig = px.box(adidas_sales_df, x='Units Sold', y='Product', color='Brand')
fig.show()

In [ ]:
# Identify outliers using IQR (Interquartile Range)
Q1 = adidas_sales_df['Units Sold'].quantile(0.25)
Q3 = adidas_sales_df['Units Sold'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers (adjust the method based on your preference)
adidas_sales_df_filtered = adidas_sales_df[(adidas_sales_df['Units Sold'] >= lower_bound) & (adidas_sales_df['Units Sold'] <= upper_bound)]

adidas_sales_df_filtered.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9130 entries, 0 to 9636
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Retailer        9130 non-null   object        
 1   Invoice Date    9130 non-null   datetime64[ns]
 2   Region          9130 non-null   object        
 3   State           9130 non-null   object        
 4   Product         9130 non-null   object        
 5   Price per Unit  9130 non-null   int64         
 6   Units Sold      9130 non-null   int64         
 7   Total Sales     9130 non-null   int64         
 8   Sales Method    9130 non-null   object        
 9   Brand           9130 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 784.6+ KB


In [ ]:
# Identify outliers using IQR (Interquartile Range)
Q1 = nike_sales_df['Units Sold'].quantile(0.25)
Q3 = nike_sales_df['Units Sold'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers (adjust the method based on your preference)
nike_sales_df_filtered = nike_sales_df[(nike_sales_df['Units Sold'] >= lower_bound) & (nike_sales_df['Units Sold'] <= upper_bound)]

nike_sales_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8856 entries, 70 to 9359
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Invoice Date    8856 non-null   datetime64[ns]
 1   Product         8856 non-null   object        
 2   Region          8856 non-null   object        
 3   Retailer        8856 non-null   object        
 4   Sales Method    8856 non-null   object        
 5   State           8856 non-null   object        
 6   Price per Unit  8856 non-null   int64         
 7   Total Sales     8856 non-null   int64         
 8   Units Sold      8856 non-null   int64         
 9   Brand           8856 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 761.1+ KB


In [ ]:
print(adidas_sales_df.describe())
print(adidas_sales_df_filtered.describe())

                        Invoice Date  Price per Unit   Units Sold  \
count                           9637     9637.000000  9637.000000   
mean   2021-05-10 16:52:11.929023232       45.218948   256.886064   
min              2020-01-01 00:00:00        7.000000     6.000000   
25%              2021-02-17 00:00:00       35.000000   106.000000   
50%              2021-06-04 00:00:00       45.000000   176.000000   
75%              2021-09-16 00:00:00       55.000000   350.000000   
max              2021-12-31 00:00:00      110.000000  1275.000000   
std                              NaN       14.704520   214.097600   

        Total Sales  
count   9637.000000  
mean   12451.621355  
min      160.000000  
25%     4068.000000  
50%     7805.000000  
75%    15834.000000  
max    82500.000000  
std    12708.094249  
                        Invoice Date  Price per Unit   Units Sold  Total Sales
count                           9130     9130.000000  9130.000000   9130.00000
mean   2021-05-16 04:1

In [ ]:
print(adidas_sales_df['Total Sales'].sum())
print(nike_sales_df['Total Sales'].sum())

119996275
8629275


In [ ]:
adidas_sales_df[adidas_sales_df['Units Sold']<300].describe()

,Invoice Date,Price per Unit,Units Sold,Total Sales
count,6801,6801.000000,6801.000000,6801.000000
mean,2021-05-19 14:36:47.322452736,42.984855,140.600353,6161.068519
min,2020-01-01 00:00:00,7.000000,6.000000,160.000000
25%,2021-02-23 00:00:00,34.000000,87.000000,3150.000000
50%,2021-06-06 00:00:00,42.000000,135.000000,5460.000000
75%,2021-09-18 00:00:00,51.000000,192.000000,8401.000000
max,2021-12-31 00:00:00,103.000000,299.000000,23220.000000
std,NaN,13.819538,67.326729,3800.318178


In [ ]:
nike_sales_df[nike_sales_df['Units Sold']<300].describe()

,Invoice Date,Price per Unit,Total Sales,Units Sold
count,9360,9360.000000,9360.00000,9360.000000
mean,2021-05-09 08:01:59.999999744,44.739209,921.93109,25.852991
min,2020-01-01 00:00:00,7.000000,0.00000,0.000000
25%,2021-02-15 00:00:00,35.000000,42.00000,11.000000
50%,2021-06-03 00:00:00,45.000000,95.00000,18.000000
75%,2021-09-16 00:00:00,55.000000,1463.00000,35.000000
max,2021-12-31 00:00:00,110.000000,8250.00000,128.000000
std,NaN,14.506338,1413.84279,21.591922
